In [2]:
import json
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from scipy.interpolate import interp1d
from scipy.stats import pearsonr
from scipy.signal import savgol_filter

addressPrefix='C:/Users/sorush.omidvar/Google Drive/Documents/Educational/TAMU/Research/CGM Dataset/Bobak/'
if not os.path.exists(addressPrefix):
    addressPrefix='C:/GDrive/Documents/Educational/TAMU/Research/CGM Dataset/Bobak/'
pd.options.mode.chained_assignment = None  # default='warn'
plt.style.use({'figure.facecolor':'white'})

In [44]:
#Breakfast in CPF format
B1=[42.5,15,12]
B2=[170,15,12]
B3=[170,60,12]
B4=[170,15,48]
B5=[170,15,12]
B6=[170,60,48]

#Lunch in CPF format
L0=[42.5,60,12]
L1=[42.5,15,12]
L2=[170,15,12]
L3=[170,60,12]
L4=[170,15,48]
L5=[170,15,12]
L6=[170,60,48]
L7=[170,60,48]
L8=[170,60,12]
L9=[170,15,48]
L10=[42.5,60,12]
L11=[42.5,15,48]
L12=[42.5,15,12]

macroData=[]

macroData=[[30,B1,'11:54:04'],
           [30,L6,'13:00:03'],
           [31,B2,'07:52:34'],
           [31,L2,'']



           ]

[B2,L2],
[B3,L3],
[B4,L0],
[B5,L8],
[B6,L1],
[B1,L7],
[B2,L1],
[B3,L5],
[B4,L10],
[B5,L9],
[B6,L12],
[B1,L4]]
flat_list = []
for element in macroData:
    print("hashem",element)

for counter,element in enumerate(macroData):
    for subCounter,subElement in enumerate(element):
        flat_list.append([counter+30,subElement[0],subElement[1],subElement[2]])
for element in flat_list:
    print(element)


macroData=macroData.extend(macroData)

# dfMacro=pd.DataFrame

hashem [[42.5, 15, 12], [170, 60, 48]]
hashem [[170, 15, 12], [170, 15, 12]]
hashem [[170, 60, 12], [170, 60, 12]]
hashem [[170, 15, 48], [42.5, 60, 12]]
hashem [[170, 15, 12], [170, 60, 12]]
hashem [[170, 60, 48], [42.5, 15, 12]]
hashem [[42.5, 15, 12], [170, 60, 48]]
hashem [[170, 15, 12], [42.5, 15, 12]]
hashem [[170, 60, 12], [170, 15, 12]]
hashem [[170, 15, 48], [42.5, 60, 12]]
hashem [[170, 15, 12], [170, 15, 48]]
hashem [[170, 60, 48], [42.5, 15, 12]]
hashem [[42.5, 15, 12], [170, 15, 48]]
[30, 42.5, 15, 12]
[30, 170, 60, 48]
[31, 170, 15, 12]
[31, 170, 15, 12]
[32, 170, 60, 12]
[32, 170, 60, 12]
[33, 170, 15, 48]
[33, 42.5, 60, 12]
[34, 170, 15, 12]
[34, 170, 60, 12]
[35, 170, 60, 48]
[35, 42.5, 15, 12]
[36, 42.5, 15, 12]
[36, 170, 60, 48]
[37, 170, 15, 12]
[37, 42.5, 15, 12]
[38, 170, 60, 12]
[38, 170, 15, 12]
[39, 170, 15, 48]
[39, 42.5, 60, 12]
[40, 170, 15, 12]
[40, 170, 15, 48]
[41, 170, 60, 48]
[41, 42.5, 15, 12]
[42, 42.5, 15, 12]
[42, 170, 15, 48]


In [5]:
def loadingDataReader(fileAddress,parameter):
    try:
        jsonFile = open(fileAddress, )
        data = json.load(jsonFile)

        dateList=[]
        timeList=[]
        valList=[]
        for i in data:
            for j in data[i]:
                timeTemp=j["time"]
                timeTemp=int(timeTemp[0:2])*3600+int(timeTemp[3:5])*60+int(timeTemp[6:8])
                timeList.append(timeTemp)

                dateTemp=i
                dateTemp = datetime.strptime(dateTemp, '%m-%d')+relativedelta(years=+121)
                dateTemp=dateTemp.timetuple().tm_yday
                dateList.append(dateTemp)
                valList.append(float(j["value"]))
        jsonFile.close()
    except:
        print("An exception occurred")

    parList = []
    for i in range (0,len(dateList)):
        parList.append(parameter)

    return dateList,timeList,valList,parList

def loadingDataAux(fileAddress,parameter,allDate,allTime,allVal,allPar):
    fileAddress=os.path.normpath(fileAddress)
    [tempDate,tempTime,tempVal,tempPar]=loadingDataReader(fileAddress,parameter)
    allDate.extend(tempDate)
    allTime.extend(tempTime)
    allVal.extend(tempVal)
    allPar.extend(tempPar)
    print(parameter," is done")

    return allDate,allTime,allVal,allPar

def dataResEnhancer(df):
    gapLength=1
    df=df.sort_values(by=['Parameter', 'Date','Time'])
    gapList=[]

    highResSensor=['Acc','EDA','HR','Temp','CGM']
    for sensor in highResSensor:
        newQuery=df[df['Parameter']==sensor]
        for counter in range(0,len(newQuery)-1):
            if newQuery.iloc[counter+1,0]==newQuery.iloc[counter+1,0]:
                if newQuery.iloc[counter+1,1]-newQuery.iloc[counter,1]>gapLength:
                    gapList.append([newQuery.iloc[counter,0],newQuery.iloc[counter,1],newQuery.iloc[counter+1,1],newQuery.iloc[counter,3]])
    appendedData=[]
    for element in gapList:
        duration= element[2]-element[1]
        if duration%gapLength==0:
            fillerNumber=int(duration/gapLength-1)
        else:
            fillerNumber=int(duration/gapLength)
        for counter in range(0,fillerNumber):
            appendedData.append([element[0],element[1]+(counter+1)*gapLength,float('nan'),element[3]])

    df = df.append(pd.DataFrame(appendedData,columns=['Date','Time','Value','Parameter']),ignore_index = True)
    df=df.sort_values(by=['Parameter','Date','Time'])

    df=df.interpolate(method='linear')
    return df

def dataCleaner(df):
    i = df[df.Date == 30].index #first data and noisy
    df=df.drop(i)

    i = df[df.Date == 40].index #Partial E4
    df=df.drop(i)

    i = df[df.Date == 41].index #No E4
    df=df.drop(i)

    i = df[df.Date >= 42].index #CGM becomes too noisy at the end of the study
    df=df.drop(i)

    return df

def dataAmputator(df):
    amputations=[[31,38290,39010],[31,61937,62638],[31,66226,71099],
              [32,58667,59563],[32,63875,63902],
              [33,25127,34731],[33,57996,58812],
              [34,25556,30763],[34,58736,59877],[34,66069,66092],
              [35,27945,31416],[35,57283,58147],
              [36,29865,33926],[36,59231,60096],
              [37,26911,32544],[37,40214,41380],
              [38,26707,34560],[38,38172,39273],
              [39,31053,35335],[39,57408,58655]]

    for amputation in amputations:
        i=df[(df['Date']==amputation[0]) & (df['Time']>=amputation[1]) & (df['Time']<=amputation[2])].index
        df.loc[i,'Value']=float('nan')
        # df=df.drop(i)

    return df

def loadingData(addressPrefix):
    if not os.path.exists(addressPrefix+'Result-interpolated.csv'):
        allDate=[]
        allTime=[]
        allVal=[]
        allPar=[]

        sensorList=[['intraday-ACC_E4.json','Acc'],['intraday-calories.json','Cal'],
                    ['intraday-EDA_E4.json','EDA'],['intraday-HR_E4.json','HR'],
                    ['intraday-steps.json','Step'],['intraday-TEMP_E4.json','Temp'],
                    ['intraday-glucose.json','CGM']]

        for element in sensorList:
            fileAddress=addressPrefix+element[0]
            allDate,allTime,allVal,allPar=loadingDataAux(fileAddress,element[1],allDate,allTime,allVal,allPar)

        dfOriginal = pd.DataFrame(list(zip(allDate,allTime,allVal,allPar)),
                          columns =['Date','Time','Value','Parameter'])
        dfOriginal=dfOriginal.sort_values(by=['Parameter','Date','Time'])
        dfOriginal.to_csv(addressPrefix+'Result-original.csv', header=True,index=False)
        print("Result-Original is saved")
        dfInterp=dataCleaner(dfOriginal)
        dfInterp=dataResEnhancer(dfInterp)
        dfInterp=dataAmputator(dfInterp)
        dfInterp.to_csv(addressPrefix+'Result-interpolated.csv', header=True,index=False)
        print("Result-Interpolated is saved")
    else:
        dfOriginal=pd.read_csv(addressPrefix+'Result-original.csv')
        dfInterp=pd.read_csv(addressPrefix+'Result-interpolated.csv')
    return dfOriginal,dfInterp

dfOriginal,dfInterp=loadingData(addressPrefix)

In [24]:
def STMI_XGBoost(trainData,testData,trainLabels,testLabels,patientName,testDay):
    accuracyBest=-1
    confMatrixBest=[]
    for threshold in np.arange(0.4,0.8,0.2):
        for eta in np.arange(0.05,0.50,0.05):
            for maxDepth in np.arange(3,10):
                clf = XGBClassifier(eta=eta,max_depth=maxDepth,objective = "binary:logistic",eval_metric = "logloss",use_label_encoder =False)
                clf.fit(trainData, trainLabels)
                slidingWindowPrediction = clf.predict_proba(testData)
                slidingWindowPrediction=slidingWindowPrediction[:,1]
                slidingWindowPrediction[slidingWindowPrediction>=threshold]=1
                slidingWindowPrediction[slidingWindowPrediction<threshold]=0

                confMatrix=sklearn.metrics.confusion_matrix(testLabels,slidingWindowPrediction)
                accuracy=sklearn.metrics.accuracy_score(testLabels,slidingWindowPrediction)

                if accuracy>accuracyBest:
                    confMatrixBest=confMatrix
                    accuracyBest=accuracy
                    threshBest=threshold
                    modelBest=clf

    features=np.round(modelBest.feature_importances_*100,1)
    # saveModel('XG',modelBest,threshBest,patientName,testDay)

    return modelBest,confMatrixBest,threshBest,features

def featureExtractor(df):
    featureList=[]
    sensors=['EDA', 'Step', 'Acc', 'CGM', 'Cal', 'Temp', 'HR']
    for sensor in sensors:
        varData=df[df['Parameter']==sensor]
        varTime=varData['Time'].tolist()
        varData=varData['Value'].tolist()

        varTime=np.asarray(varTime)
        varData=np.asarray(varData)
        featureList.append(np.mean(varData))
        featureList.append(np.median(varData))
        featureList.append(np.mod(varData))
        featureList.append(np.max(varData))
        featureList.append(np.min(varData))
        featureList.append(np.var(varData))

        featureList.append(np.mean(np.diff(varData,n=1)))
        featureList.append(np.median(np.diff(varData,n=1)))
        featureList.append(np.mod(np.diff(varData,n=1)))
        featureList.append(np.max(np.diff(varData,n=1)))
        featureList.append(np.min(np.diff(varData,n=1)))
        featureList.append(np.var(np.diff(varData,n=1)))

        featureList.append(np.mean(np.diff(varData,n=2)))
        featureList.append(np.median(np.diff(varData,n=2)))
        featureList.append(np.mod(np.diff(varData,n=2)))
        featureList.append(np.max(np.diff(varData,n=2)))
        featureList.append(np.min(np.diff(varData,n=2)))
        featureList.append(np.var(np.diff(varData,n=2)))

        featureList.append(np.trapz(varData))

    #TALK TO HOOMAN ON WHAT TO EXTRACT FROM CGM SIGNALS
    #REDO IT WITH FITBIT AND E4
    #TRAIN ON 5 DAYS AND TEST ON THE OTHER 5
    #use XGBoost
    #GET THE MACRO FROM BOBAK AND PUT IT THERE
    #FOR MEAL TO MEAL PERSPECTIVE, MAKE THE WHOLE DAY INTO 3 CHUNCKS (MEAL TO MEAL)
    #PLOT DAY-DAY VS MEAL TO MEAL AND SHOW THEM HOW WE ARE SEAPRATING THINGS (CGM,EDA,...)
    #ADD AUC FOR ALL OF THE PARAMETERS


    #https://github.com/fraunhoferportugal/tsfel/tree/9319db4368303cf10adb3aeb72cd4235a8085307


    return featureList


def testTrainSplitFuncAux(df):
    dataList=[]
    days=list(set(df['Date']))
    for day in days:
        for hourCounter in range(0,1):
            startHour=hourCounter
            endHour=startHour+24
            dfTemp=df[df['Date']==day]
            dfTemp=dfTemp[dfTemp['Time']>=startHour*3600]
            dfTemp=dfTemp[dfTemp['Time']<=endHour*3600]
            dataList.append(featureExtractor(dfTemp))
        break

    return dataList

def testTrainSplitFunc(dfSensor, dfMacro):
    days=list(set(df['Date']))
    random.shuffle(days)
    daysTrain=days[0:6]
    daysTest=days[6:9]
    dfTrain = df[df['Date'].isin(daysTrain)]
    dfTest = df[df['Date'].isin(daysTest)]

    trainList=testTrainSplitFuncAux(dfTrain)
    testList=testTrainSplitFuncAux(dfTest)

    return trainList, testList

a,b=testTrainSplitFunc(dfInterp)

['EDA', 'Step', 'Acc', 'CGM', 'Cal', 'Temp', 'HR']


TypeError: list.append() takes exactly one argument (0 given)

In [ ]:

windowDuration=2
lagTimes=np.arange(-2,2.5,0.5)
mealTimes=[12.33,12.33,12.33,12.16,11.92,12.5,12.16,11.33,12.25]
plotFlag=False
plotParamter='Temp'

pearsonRatios=[]
days=dfInterp['Date']
days=days.values.tolist()
days=set(days)
days=list(days)
days.sort()
pearsonRatios=np.zeros((len(lagTimes),len(days)))
for i,lagTime in enumerate(lagTimes):
    for counter,day in enumerate(days):
        mealTime=mealTimes[counter]
        queryCGM=dfInterp[dfInterp['Date']==day]
        queryCGM['Time']=queryCGM['Time']/3600
        queryCGM=queryCGM[queryCGM['Parameter']=='CGM']
        queryCGM=queryCGM[queryCGM['Time']<=(mealTime+windowDuration)]
        queryCGM=queryCGM[queryCGM['Time']>=mealTime]

        queryVar=dfInterp[dfInterp['Date']==day]
        queryVar['Time']=queryVar['Time']/3600
        queryVar=queryVar[queryVar['Parameter']==plotParamter]
        queryVar=queryVar[queryVar['Time']<=(mealTime+windowDuration+lagTime)]
        queryVar=queryVar[queryVar['Time']>=(mealTime+lagTime)]
        if plotFlag:
            fig, ax1 = plt.subplots ()
            ax2 = ax1.twinx ()

            queryCGM.plot(ax=ax1,x='Time',y='Value',color='green')
            ax1.tick_params (axis='y', labelcolor='green')

            queryVar.plot(ax=ax2,x='Time',y='Value',color='red')
            ax2.tick_params (axis='y', labelcolor='red')

            ax1.set_ylabel('CGM')
            ax1.set_xlabel('Day')
            ax1.yaxis.label.set_color("green")

            ax2.set_ylabel(plotParamter)
            ax2.yaxis.label.set_color("red")

            # plt.savefig(addressPrefix+'figure'+str(day)+'.jpg')
            plt.title("Day="+str(day))
            ax1.get_legend().remove()
            ax2.get_legend().remove()
            plt.show()
        CGMData=queryCGM['Value'].values.tolist()
        CGMData=np.asarray(CGMData)
        varData=queryVar['Value'].values.tolist()
        varData=np.asarray(varData)
        discardIndex=np.argwhere(np.isnan(varData))
        varData=np.delete(varData,discardIndex)
        CGMData=np.delete(CGMData,discardIndex)
        corr, pval = pearsonr(CGMData,varData)
        # pearsonRatios.append([lagTime,day,np.round(corr,3)])
        pearsonRatios[i,counter]=np.round(corr,3)

for i,lagTime in enumerate(lagTimes):
    print("Lag time=",lagTime,"Mean pearson=",np.round(np.mean(pearsonRatios[i,:]),3))
    print(pearsonRatios[i,:])
    print("----------------------")
plt.boxplot(pearsonRatios)
plt.xticks(np.arange(1,len(lagTimes)+1), lagTimes)
plt.ylabel('Pearson Corr')
plt.xlabel('Lag between CGM and Var [hr]')
plt.show()